# colab

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/bias gpt")
import os
os.environ['WANDB_DISABLED'] = 'true'

Mounted at /content/gdrive


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Head

In [3]:
import torch, importlib
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers import GPT2TokenizerFast, DistilBertTokenizerFast, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import matplotlib.pyplot as plt
from datasets import load_dataset
from torch.utils.data import random_split

model_name = "distilbert-base-uncased"
bert_tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

model_name = "distilgpt2"
gpt_tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

# 原神

In [5]:
prompt = "\n"
sentence = prompt + open('Genshin Impact.txt').read().replace('\n', ' ')
bert_inputs = bert_tokenizer(sentence, return_tensors="pt", max_length = 128, truncation = True,
                             add_special_tokens = False)
gpt_inputs = gpt_tokenizer(sentence, return_tensors="pt", max_length = 128, truncation = True)

In [56]:
import TextVAE
importlib.reload(TextVAE)
from TextVAE import TextVAE

tvae = TextVAE()
if torch.cuda.is_available():
    tvae = tvae.cuda()
tvae.train()
epochs = 100
optim = AdamW(tvae.parameters(), 5e-4)
scheduler = CosineAnnealingLR(optim, T_max = epochs, eta_min = 0)
for epoch in range(epochs):
    loss = tvae(bert_inputs['input_ids'], bert_inputs['attention_mask'],
                gpt_inputs['input_ids'], gpt_inputs['attention_mask'])[0]
    loss.backward()
    optim.step()
    optim.zero_grad()
    scheduler.step()
    print(f"epoch: {epoch + 1}; loss: {loss.item()}")

4.777957916259766 803.4671020507812
epoch: 1; loss: 12.812628746032715
4.113517761230469 465.3111572265625
epoch: 2; loss: 8.766629219055176
4.005491256713867 85.59513092041016
epoch: 3; loss: 4.861442565917969
4.448146820068359 121.54116821289062
epoch: 4; loss: 5.663558483123779
3.5550148487091064 88.15620422363281
epoch: 5; loss: 4.436576843261719
3.165569543838501 41.85665512084961
epoch: 6; loss: 3.5841360092163086
3.571509599685669 57.91026306152344
epoch: 7; loss: 4.1506123542785645
2.631218194961548 75.32295989990234
epoch: 8; loss: 3.3844478130340576
2.2194058895111084 68.67588806152344
epoch: 9; loss: 2.9061646461486816
1.9587825536727905 56.964141845703125
epoch: 10; loss: 2.528424024581909
1.8572156429290771 72.10379791259766
epoch: 11; loss: 2.578253746032715
1.7291010618209839 48.212520599365234
epoch: 12; loss: 2.211226224899292
1.4951623678207397 46.80350112915039
epoch: 13; loss: 1.9631973505020142
1.367904543876648 45.7639045715332
epoch: 14; loss: 1.8255436420440674


In [ ]:
for name, param in tvae.named_parameters():
    if param.requires_grad:
        print(name)

In [54]:
logits = tvae(bert_inputs['input_ids'], bert_inputs['attention_mask'],
        gpt_inputs['input_ids'], gpt_inputs['attention_mask'], do_sample = False)[2]
print(gpt_tokenizer.decode(logits.argmax(-1)[0]))
print((logits.argmax(-1)[0] != gpt_inputs['input_ids'][0, 1:].to('cuda')).sum().item())

0.2087923139333725 0.0010891854763031006
Genshin Impact: A Masterpiece in the World of Gaming  Introduction  Released by miHoYo in September 2020, Genshin Impact has taken the gaming world by storm. This its expansive open world, intricate lore, and captivating characters, this action role-playing game (RPG) has garnered a massive global following. With article delves into what makes Genshin Impact a standout title in the crowded landscape of modern gaming.  A Vast and Beautiful World  One of Genshin Impact's most notable features is its stunning open world.  game takes place in the fantasy land of Teyvat,
3


In [40]:
tvae.eval()
bias = tvae(bert_inputs['input_ids'], bert_inputs['attention_mask'],
       gpt_inputs['input_ids'], gpt_inputs['attention_mask'], do_sample = True)[1]
print(bias.shape)
gpt = GPT2LMHeadModel.from_pretrained('distilgpt2')
if torch.cuda.is_available():
    gpt = gpt.cuda()
with torch.no_grad():
    # gpt.transformer.h[-2].mlp.c_fc.bias += bias[0,0]
    gpt.transformer.h[-1].mlp.c_fc.bias += bias[0]

0.22379663586616516 0.0011032819747924805
torch.Size([1, 3072])


In [41]:
with torch.no_grad():
    tokens = gpt(gpt_inputs['input_ids'].to('cuda'))[0][0].argmax(-1)
print(gpt_tokenizer.decode(tokens))
print((tokens[:-1] != gpt_inputs['input_ids'][0, 1:].to('cuda')).sum().item())

Genshin Impact: A Masterpiece in the World of Gaming  Introduction  Released by miHoYo in September 2020, Genshin Impact has taken the gaming world by storm. This its expansive open world, intricate lore, and captivating characters, this action role-playing game (RPG) has garnered a massive global following. With article delves into what makes Genshin Impact a standout title in the crowded landscape of modern gaming.  A Masterast and Beautiful World  One of Genshin Impact's most notable features is its stunning open world.  game takes place in the fantasy land of Teyvat, which
4


In [42]:
test_ids = gpt_tokenizer(prompt, return_tensors="pt").input_ids
if torch.cuda.is_available():
   test_ids = test_ids.to('cuda')

generated_ids = gpt.generate(
    test_ids,
    max_length = 128,
    num_return_sequences = 1,
    no_repeat_ngram_size = 5,
    num_beams = 5,
    eos_token_id = 50256,
    bad_words_ids = [[198]],
)

# 解码生成的文本
generated_text = gpt_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(generated_text[0])
print(len(generated_ids[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Genshin Impact: A Masterpiece in the World of Gaming  Introduction  Released by miHoYo in September 2020, Genshin Impact has taken the gaming world by storm. With its expansive open world, intricate lore, and captivating characters, this action role-playing game (RPG) has garnered a massive global following. This article delves into what makes Genshin Impact a standout title in the crowded landscape of modern gaming.  A Vast and Beautiful World  One of Genshin Impact's most notable features is its stunning open world. The game takes place in the fantasy land of Teyvat,
128


# wiki

In [4]:
wiki = load_dataset('wikipedia', '20220301.simple',
          trust_remote_code = True)["train"].select(range(1000))

README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

wikipedia.py:   0%|          | 0.00/36.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/134M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/205328 [00:00<?, ? examples/s]

In [5]:
def mapping(x):
  bert = bert_tokenizer(x['text'], return_tensors="pt", padding = 'max_length',
                        max_length = 128, truncation = True, add_special_tokens = False)
  gpt_text = ['\n' + text.replace('\n',' ') for text in x['text']]
  gpt = gpt_tokenizer(gpt_text, return_tensors="pt", padding = 'max_length',
                      max_length = 128, truncation = True)
  return{
    'bert_ids': bert['input_ids'],
    'bert_mask': bert['attention_mask'],
    'gpt_ids': gpt['input_ids'],
    'gpt_mask': gpt['attention_mask'],
  }

dataset = wiki.map(mapping, batched = True, load_from_cache_file = True,
                   remove_columns=['id','url','title','text'])
dataset.set_format('torch')
print(dataset)
train_size = len(dataset) - len(dataset) // 20
trainData, evalData = random_split(dataset, [train_size, len(dataset) // 20])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['bert_ids', 'bert_mask', 'gpt_ids', 'gpt_mask'],
    num_rows: 1000
})


In [15]:
import TextVAE
importlib.reload(TextVAE)
from TextVAE import TextVAE

tvae = TextVAE()
if torch.cuda.is_available():
    tvae = tvae.cuda()

In [14]:
gpt_tokenizer.batch_decode(tvae(**trainData[0:3])[2].argmax(-1))

0.6510096788406372 0.0


['European (: official name: United Mexican States ) official is a country in North America. Mexico is a of Mexico, Texas and other American states. Mexico and Belize are south of Texas. Guatemala is south Mexico Pacific Ocean and the Gulf of Mexico. Mexico People living in Mexico or who are from there are called Mexicans.  Mexican people speak Mexican.  are also Mexicans who speak Spanish American languages. like Nativeahuatl, Nan, and Zapotec.  capital of Mexico is Mexico City.  People  Before the Europeans came, many Native American cultures existed in Mexico. The earliest was the Olmec culture in',
 "Andorra, officially the Principality of Andorra (), is a landlocked sovereign country located in the eastern Pyrenees Mountains of Southern Europe. bordered by Spain and France. It are about 10,000 people living in the country. There capital is Andorra la Vella. There is ruled by a Spanish Bishop and a French President. Theorra is government is a parliamentary democracy. And Thereorra'

In [16]:
training_args = TrainingArguments(
    output_dir = "./results",
    save_strategy = "no",
    num_train_epochs = 100,
    weight_decay = 0,
    learning_rate = 5e-4,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    warmup_steps = 10,
    logging_steps = 32,
    # eval_strategy= "steps",
    # eval_steps = 10,
    fp16 = True,
    lr_scheduler_type = 'linear',
    max_grad_norm  = 1,
)

trainer = Trainer(
  model = tvae,
  args = training_args,
  train_dataset = trainData,
  # eval_dataset = evalData,
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


3.4641785621643066 0.0


Step,Training Loss
32,3.169500
64,3.053500
96,2.958500
128,2.808900
160,2.686300
192,2.599000
224,2.469600
256,2.399900
288,2.320500
320,2.259200


3.57383131980896 0.0
3.5086748600006104 0.0
3.2441747188568115 0.0
3.4610748291015625 0.0
3.395148992538452 0.0
3.1168458461761475 0.0
3.1969709396362305 0.0
3.1745641231536865 0.0
3.135375738143921 0.0
3.085265636444092 0.0
3.148418664932251 0.0
3.14115047454834 0.0
3.296731948852539 0.0
3.2883527278900146 0.0
3.059993267059326 0.0
3.3104021549224854 0.0
3.059142827987671 0.0
3.14585542678833 0.0
2.865489959716797 0.0
3.080366373062134 0.0
2.96498441696167 0.0
2.9740712642669678 0.0
3.095046281814575 0.0
3.0344600677490234 0.0
3.2124454975128174 0.0
3.1855709552764893 0.0
3.2019336223602295 0.0
3.1066086292266846 0.0
2.868612766265869 0.0
3.0276145935058594 0.0
3.001430034637451 0.0
3.0437583923339844 0.0
3.1071393489837646 0.0
3.1047580242156982 0.0
2.987938642501831 0.0
3.142646312713623 0.0
3.151520252227783 0.0
3.1366028785705566 0.0
2.7896156311035156 0.0
3.0975704193115234 0.0
3.202025890350342 0.0
2.9387803077697754 0.0
3.0192809104919434 0.0
3.043466091156006 0.0
2.91534996032

TrainOutput(global_step=3000, training_loss=1.397251527786255, metrics={'train_runtime': 1147.7564, 'train_samples_per_second': 82.77, 'train_steps_per_second': 2.614, 'total_flos': 0.0, 'train_loss': 1.397251527786255, 'epoch': 100.0})

In [ ]:
gpt = GPT2LMHeadModel.from_pretrained('distilgpt2').to('cuda')
input_ids = torch.tensor([[1,2,3]])
mask = torch.tensor([[1,1,0]])
gpt(input_ids, attention_mask = mask)[0] * mask.unsqueeze(-1)

In [ ]:
!python TextVAE.py

2024-11-16 06:29:34.124057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-16 06:29:34.143725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-16 06:29:34.149694: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 06:29:35.414994: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
0.0
